**Building Model**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Embedding
from keras.layers.core import Dense, SpatialDropout1D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping


Mounted at /content/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


USING REDDIT DATASET TO FIND THE TYPE OF DEPRESSION THE USER IS SUFFERING FROM

In [ ]:
reddit_train_df = pd.read_csv('/content/drive/My Drive/reddit_train.csv',encoding='mac_roman')
reddit_train_df.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,lex_liwc_WPS,lex_liwc_Sixltr,lex_liwc_Dic,lex_liwc_function,lex_liwc_pronoun,lex_liwc_ppron,lex_liwc_i,lex_liwc_we,lex_liwc_you,lex_liwc_shehe,lex_liwc_they,lex_liwc_ipron,lex_liwc_article,lex_liwc_prep,lex_liwc_auxverb,lex_liwc_adverb,lex_liwc_conj,lex_liwc_negate,lex_liwc_verb,lex_liwc_adj,lex_liwc_compare,lex_liwc_interrog,lex_liwc_number,lex_liwc_quant,lex_liwc_affect,...,lex_liwc_motion,lex_liwc_space,lex_liwc_time,lex_liwc_work,lex_liwc_leisure,lex_liwc_home,lex_liwc_money,lex_liwc_relig,lex_liwc_death,lex_liwc_informal,lex_liwc_swear,lex_liwc_netspeak,lex_liwc_assent,lex_liwc_nonflu,lex_liwc_filler,lex_liwc_AllPunc,lex_liwc_Period,lex_liwc_Comma,lex_liwc_Colon,lex_liwc_SemiC,lex_liwc_QMark,lex_liwc_Exclam,lex_liwc_Dash,lex_liwc_Quote,lex_liwc_Apostro,lex_liwc_Parenth,lex_liwc_OtherP,lex_dal_max_pleasantness,lex_dal_max_activation,lex_dal_max_imagery,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,116,72.64,15.04,89.26,1.00,29.00,12.93,87.07,56.03,16.38,12.07,9.48,0.00,0.86,1.72,0.0,4.31,3.45,19.83,7.76,5.17,4.31,1.72,16.38,6.03,3.45,0.86,1.72,1.72,8.62,...,0.86,10.34,6.03,0.86,0.00,0.00,0.00,2.59,0.0,0.86,0.86,0.00,0.00,0.0,0.0,21.55,9.48,3.45,0.86,0.86,0.0,0.0,0.0,5.17,1.72,0.00,0.00,2.8571,2.6250,3.0,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,109,79.08,76.85,56.75,98.18,27.25,21.10,87.16,48.62,11.93,7.34,1.83,2.75,2.75,0.00,0.0,4.59,8.26,13.76,6.42,3.67,8.26,0.92,15.60,2.75,0.92,0.92,2.75,0.92,5.50,...,2.75,10.09,1.83,11.01,0.00,0.00,0.92,0.00,0.0,1.83,0.00,0.92,0.00,0.0,0.0,14.68,4.59,2.75,0.00,0.00,0.0,0.0,0.0,0.00,2.75,0.92,3.67,3.0000,2.8889,3.0,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,167,33.80,76.38,86.24,25.77,33.40,17.37,91.02,61.68,25.15,16.17,8.98,1.80,1.80,2.99,0.6,8.98,5.39,12.57,10.18,1.80,5.99,1.20,20.96,1.20,0.60,0.60,1.20,1.80,2.40,...,4.79,5.99,5.39,0.00,1.20,0.60,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,10.78,2.40,3.59,0.00,0.00,0.6,0.0,0.0,2.40,1.80,0.00,0.00,2.7143,3.0000,3.0,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,273,2.98,15.25,95.42,79.26,54.60,8.06,98.90,65.57,30.40,23.44,16.12,0.37,0.37,6.59,0.0,6.96,3.30,9.16,8.79,6.59,9.89,3.66,20.88,3.66,1.83,1.10,0.00,1.10,8.79,...,1.83,3.30,6.23,0.73,0.37,0.37,0.00,0.00,0.0,2.56,0.00,0.73,0.73,0.0,0.0,12.09,2.56,7.33,0.00,0.00,0.0,0.0,0.0,1.47,0.00,0.73,0.00,3.0000,2.6364,3.0,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,89,32.22,28.71,84.01,1.00,17.80,31.46,88.76,52.81,15.73,11.24,7.87,0.00,0.00,3.37,0.0,4.49,4.49,8.99,13.48,4.49,4.49,2.25,13.48,4.49,2.25,1.12,1.12,1.12,7.87,...,2.25,2.25,10.11,1.12,0.00,2.25,1.12,0.00,0.0,1.12,0.00,0.00,0.00,0.0,0.0,16.85,5.62,6.74,1.12,0.00,0.0,0.0,0.0,0.00,1.12,2.25,0.00,3.0000,3.0000,3.0,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [ ]:
reddit_train_df['subreddit'].value_counts()

ptsd                584
relationships       552
anxiety             503
domesticviolence    316
assistance          289
survivorsofabuse    245
homeless            168
almosthomeless       80
stress               64
food_pantry          37
Name: subreddit, dtype: int64

PREPROCESSING

In [ ]:
reddit_train_df = reddit_train_df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):    
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text


reddit_train_df['text'] = reddit_train_df['text'].apply(clean_text)
reddit_train_df['text'] = reddit_train_df['text'].str.replace('\d+', '')


In [ ]:
reddit_train_df.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,lex_liwc_WPS,lex_liwc_Sixltr,lex_liwc_Dic,lex_liwc_function,lex_liwc_pronoun,lex_liwc_ppron,lex_liwc_i,lex_liwc_we,lex_liwc_you,lex_liwc_shehe,lex_liwc_they,lex_liwc_ipron,lex_liwc_article,lex_liwc_prep,lex_liwc_auxverb,lex_liwc_adverb,lex_liwc_conj,lex_liwc_negate,lex_liwc_verb,lex_liwc_adj,lex_liwc_compare,lex_liwc_interrog,lex_liwc_number,lex_liwc_quant,lex_liwc_affect,...,lex_liwc_motion,lex_liwc_space,lex_liwc_time,lex_liwc_work,lex_liwc_leisure,lex_liwc_home,lex_liwc_money,lex_liwc_relig,lex_liwc_death,lex_liwc_informal,lex_liwc_swear,lex_liwc_netspeak,lex_liwc_assent,lex_liwc_nonflu,lex_liwc_filler,lex_liwc_AllPunc,lex_liwc_Period,lex_liwc_Comma,lex_liwc_Colon,lex_liwc_SemiC,lex_liwc_QMark,lex_liwc_Exclam,lex_liwc_Dash,lex_liwc_Quote,lex_liwc_Apostro,lex_liwc_Parenth,lex_liwc_OtherP,lex_dal_max_pleasantness,lex_dal_max_activation,lex_dal_max_imagery,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)",said felt way suggeted go rest trigger ahead y...,33181,1,0.8,1521614353,5,1.806818,116,72.64,15.04,89.26,1.00,29.00,12.93,87.07,56.03,16.38,12.07,9.48,0.00,0.86,1.72,0.0,4.31,3.45,19.83,7.76,5.17,4.31,1.72,16.38,6.03,3.45,0.86,1.72,1.72,8.62,...,0.86,10.34,6.03,0.86,0.00,0.00,0.00,2.59,0.0,0.86,0.86,0.00,0.00,0.0,0.0,21.55,9.48,3.45,0.86,0.86,0.0,0.0,0.0,5.17,1.72,0.00,0.00,2.8571,2.6250,3.0,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)",hey r assistance sure right place post goes im...,2606,0,1.0,1527009817,4,9.429737,109,79.08,76.85,56.75,98.18,27.25,21.10,87.16,48.62,11.93,7.34,1.83,2.75,2.75,0.00,0.0,4.59,8.26,13.76,6.42,3.67,8.26,0.92,15.60,2.75,0.92,0.92,2.75,0.92,5.50,...,2.75,10.09,1.83,11.01,0.00,0.00,0.92,0.00,0.0,1.83,0.00,0.92,0.00,0.0,0.0,14.68,4.59,2.75,0.00,0.00,0.0,0.0,0.0,0.00,2.75,0.92,3.67,3.0000,2.8889,3.0,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",mom hit newspaper shocked would knows dont lik...,38816,1,0.8,1535935605,2,7.769821,167,33.80,76.38,86.24,25.77,33.40,17.37,91.02,61.68,25.15,16.17,8.98,1.80,1.80,2.99,0.6,8.98,5.39,12.57,10.18,1.80,5.99,1.20,20.96,1.20,0.60,0.60,1.20,1.80,2.40,...,4.79,5.99,5.39,0.00,1.20,0.60,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,10.78,2.40,3.59,0.00,0.00,0.6,0.0,0.0,2.40,1.80,0.00,0.00,2.7143,3.0000,3.0,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]",met new boyfriend amazing kind sweet good stud...,239,1,0.6,1516429555,0,2.667798,273,2.98,15.25,95.42,79.26,54.60,8.06,98.90,65.57,30.40,23.44,16.12,0.37,0.37,6.59,0.0,6.96,3.30,9.16,8.79,6.59,9.89,3.66,20.88,3.66,1.83,1.10,0.00,1.10,8.79,...,1.83,3.30,6.23,0.73,0.37,0.37,0.00,0.00,0.0,2.56,0.00,0.73,0.73,0.0,0.0,12.09,2.56,7.33,0.00,0.00,0.0,0.0,0.0,1.47,0.00,0.73,0.00,3.0000,2.6364,3.0,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",october domestic violence awareness month dome...,1421,1,0.8,1539809005,24,7.554238,89,32.22,28.71,84.01,1.00,17.80,31.46,88.76,52.81,15.73,11.24,7.87,0.00,0.00,3.37,0.0,4.49,4.49,8.99,13.48,4.49,4.49,2.25,13.48,4.49,2.25,1.12,1.12,1.12,7.87,...,2.25,2.25,10.11,1.12,0.00,2.25,1.12,0.00,0.0,1.12,0.00,0.00,0.00,0.0,0.0,16.85,5.62,6.74,1.12,0.00,0.0,0.0,0.0,0.00,1.12,2.25,0.00,3.0000,3.0000,3.0,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


TOKENIZING AND PADDING THE TEXT TO FEED TO NEURAL NETWORKS

In [ ]:
MAX_NB_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 600
# This is fixed.
EMBEDDING_DIM = 512
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(reddit_train_df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 11464 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(reddit_train_df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (2838, 600)


In [ ]:
Y = pd.get_dummies(reddit_train_df['subreddit']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (2838, 10)


In [ ]:
from keras.layers.experimental.preprocessing import TextVectorization
import keras.backend as K
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Reshape,RepeatVector,LSTM,Dense,Flatten,Bidirectional,Embedding,Input,Layer,GRU,Multiply,Activation,Lambda,Dot,TimeDistributed,Dropout,Embedding
from keras.layers import Conv1D,concatenate
from keras.optimizers import Adam

In [ ]:

hidden_cnn = 264      # CNN Filters
hidden_rnn = 128      # RNN Units
batch_size = 64

MODEL BUILDING

In [ ]:
inputs = Input(shape = (X.shape[1], ))
embed_layer = Embedding(50000, 264, input_length=X.shape[1])(inputs)
embed_layer = SpatialDropout1D(0.2)(embed_layer)
lstm_layer = LSTM(100,dropout=0.2,recurrent_dropout=0.2)(embed_layer)
output = Dense(10,activation='softmax')(lstm_layer)
lstnet_model = Model(inputs = inputs, outputs = output)

In [ ]:
lstnet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs = 30
batch_size = 64
history = lstnet_model.fit(X, Y,epochs=epochs,batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/30
36/36 [==============================] - 31s 762ms/step - loss: 2.2160 - accuracy: 0.1897 - val_loss: 2.0574 - val_accuracy: 0.1919
Epoch 2/30
36/36 [==============================] - 27s 752ms/step - loss: 1.9925 - accuracy: 0.2293 - val_loss: 1.9119 - val_accuracy: 0.3099
Epoch 3/30
36/36 [==============================] - 28s 772ms/step - loss: 1.6126 - accuracy: 0.4908 - val_loss: 1.7729 - val_accuracy: 0.3521
Epoch 4/30
36/36 [==============================] - 28s 765ms/step - loss: 1.0669 - accuracy: 0.6289 - val_loss: 1.8466 - val_accuracy: 0.3768
Epoch 5/30
36/36 [==============================] - 27s 763ms/step - loss: 0.6511 - accuracy: 0.8056 - val_loss: 2.0889 - val_accuracy: 0.3556
Epoch 6/30
36/36 [==============================] - 28s 766ms/step - loss: 0.3513 - accuracy: 0.8950 - val_loss: 2.2694 - val_accuracy: 0.3486


TEST DATA

In [ ]:
reddit_test_df = pd.read_csv('/content/drive/My Drive/reddit_test.csv',encoding='mac_roman')
reddit_test_df.head()

,id,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,lex_liwc_WPS,lex_liwc_Sixltr,lex_liwc_Dic,lex_liwc_function,lex_liwc_pronoun,lex_liwc_ppron,lex_liwc_i,lex_liwc_we,lex_liwc_you,lex_liwc_shehe,lex_liwc_they,lex_liwc_ipron,lex_liwc_article,lex_liwc_prep,lex_liwc_auxverb,lex_liwc_adverb,lex_liwc_conj,lex_liwc_negate,lex_liwc_verb,lex_liwc_adj,lex_liwc_compare,lex_liwc_interrog,lex_liwc_number,lex_liwc_quant,lex_liwc_affect,...,lex_liwc_motion,lex_liwc_space,lex_liwc_time,lex_liwc_work,lex_liwc_leisure,lex_liwc_home,lex_liwc_money,lex_liwc_relig,lex_liwc_death,lex_liwc_informal,lex_liwc_swear,lex_liwc_netspeak,lex_liwc_assent,lex_liwc_nonflu,lex_liwc_filler,lex_liwc_AllPunc,lex_liwc_Period,lex_liwc_Comma,lex_liwc_Colon,lex_liwc_SemiC,lex_liwc_QMark,lex_liwc_Exclam,lex_liwc_Dash,lex_liwc_Quote,lex_liwc_Apostro,lex_liwc_Parenth,lex_liwc_OtherP,lex_dal_max_pleasantness,lex_dal_max_activation,lex_dal_max_imagery,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,896,relationships,7nu7as,"[50, 55]","Its like that, if you want or not. ME: I have ...",0,0.8,1514980773,22,-1.238793,55,2.82,57.22,55.41,5.95,11.00,7.27,94.55,67.27,29.09,18.18,5.45,0.00,9.09,3.64,0.00,10.91,0.00,12.73,7.27,1.82,14.55,7.27,18.18,1.82,3.64,1.82,5.45,0.00,1.82,...,0.00,1.82,10.91,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,23.64,12.73,5.45,1.82,0.00,0.00,0.0,0.0,1.82,1.82,0.00,0.00,2.7143,2.8889,2.6,1.0000,1.2000,1.0,1.65864,1.32245,1.80264,0.63,62,-0.148707,0.000000
1,19059,anxiety,680i6d,"(5, 10)",I man the front desk and my title is HR Custom...,0,1.0,1493348050,5,7.684583,72,64.56,50.00,31.19,92.40,14.40,25.00,84.72,44.44,9.72,8.33,6.94,1.39,0.00,0.00,0.00,1.39,5.56,11.11,8.33,5.56,8.33,0.00,13.89,1.39,0.00,2.78,2.78,1.39,4.17,...,1.39,4.17,1.39,22.22,0.00,0.0,4.17,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,15.28,6.94,4.17,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,4.17,2.4444,2.8889,3.0,1.4000,1.1250,1.0,1.69133,1.69180,1.97249,1.00,2,7.398222,-0.065909
2,7977,ptsd,8eeu1t,"(5, 10)",We'd be saving so much money with this new hou...,1,1.0,1524516630,10,2.360408,148,14.79,75.05,62.88,16.15,21.14,10.81,94.59,59.46,23.65,14.19,6.08,2.03,0.68,2.70,2.70,9.46,2.70,12.16,10.14,6.08,9.46,0.68,20.27,4.73,2.70,0.68,1.35,2.03,3.38,...,2.03,8.11,6.76,0.68,0.00,0.0,2.03,0.0,0.68,2.70,0.0,0.00,0.68,0.68,0.0,17.57,16.22,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.35,0.00,0.00,2.8333,2.9091,3.0,1.1429,1.0000,1.0,1.70974,1.52985,1.86108,1.00,8,3.149288,-0.036818
3,1214,ptsd,8d28vu,"[2, 7]","My ex used to shoot back with ""Do you want me ...",1,0.5,1524018289,5,5.997000,100,32.13,38.19,79.76,12.51,20.00,9.00,92.00,64.00,28.00,17.00,12.00,1.00,2.00,2.00,0.00,11.00,4.00,15.00,6.00,6.00,4.00,2.00,20.00,2.00,2.00,3.00,0.00,2.00,3.00,...,2.00,9.00,5.00,1.00,0.00,0.0,0.00,0.0,0.00,2.00,0.0,1.00,0.00,0.00,0.0,12.00,4.00,3.00,0.00,0.00,1.00,0.0,0.0,2.00,2.00,0.00,0.00,3.0000,2.7143,2.8,1.0000,1.3000,1.0,1.72615,1.52000,1.84909,1.00,7,6.606000,-0.066667
4,1965,relationships,7r1e85,"[23, 28]",I havent said anything to him yet because Im n...,0,0.8,1516200171,138,4.649418,78,4.96,50.00,71.23,89.84,15.60,21.79,96.15,65.38,24.36,19.23,11.54,0.00,1.28,3.85,2.56,5.13,1.28,11.54,12.82,8.97,6.41,3.85,21.79,2.56,0.00,3.85,1.28,1.28,3.85,...,0.00,5.13,3.85,0.00,1.28,0.0,0.00,0.0,0.00,1.28,0.0,1.28,0.00,0.00,0.0,20.51,5.13,2.56,0.00,1.28,1.28,0.0,0.0,2.56,5.13,2.56,0.00,3.0000,2.5000,3.0,1.1250,1.1429,1.0,1.75642,1.43582,1.91725,0.84,70,4.801869,0.141667


In [ ]:
reddit_test_df = reddit_test_df.reset_index(drop=True)


def clean_text(text):    
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text


reddit_test_df['text'] = reddit_test_df['text'].apply(clean_text)
reddit_test_df['text'] = reddit_test_df['text'].str.replace('\d+', '')

In [ ]:
MAX_NB_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 600
# This is fixed.
EMBEDDING_DIM = 512
ac=[3.327,0.842]
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(reddit_test_df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
X_test = tokenizer.texts_to_sequences(reddit_test_df['text'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)
Y_test = pd.get_dummies(reddit_test_df['subreddit']).values
print('Shape of label tensor:', Y_test.shape)


Found 5583 unique tokens.
Shape of data tensor: (715, 600)
Shape of label tensor: (715, 10)


In [ ]:
accr = lstnet_model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(ac[0],ac[1]))

23/23 [==============================] - 2s 93ms/step - loss: 3.5738 - accuracy: 0.1427


NameError: ignored

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
labels=list(set(list(reddit_train_df['subreddit'].values)))
labels

In [ ]:
new_complaint = ['I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = lstnet_model.predict(padded)
print(labels[np.argmax(pred)])

SAVING THE MODEL

In [ ]:
lstnet_model.save('cmodel.h5') 

In [ ]:
from google.colab import files
files.download("cmodel.h5")

FINDING SEVERITY OF DEPRESSION USING REGRESSION.CSV

In [ ]:
regression_df = pd.read_csv('/content/drive/My Drive/regression.csv')
regression_df.head()

In [ ]:
regression_df.drop('body', axis=1, inplace=True)
regression_df.head()

PREPROCESSING

In [ ]:
import re
def remove_mentions(text):
  text = "".join([char for char in text])
  text = re.sub(r'@(\w+)', '', text, flags=re.MULTILINE)
  return text
regression_df['title'] = regression_df['title'].apply(lambda x: remove_mentions(x))

In [ ]:
import string
string.punctuation

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

regression_df['title'] = regression_df['title'].apply(lambda x: remove_punct(x))

In [ ]:
def remove_url(text):
    text  = "".join([char for char in text])
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    return text

regression_df['title'] = regression_df['title'].apply(lambda x: remove_url(x))

In [ ]:
import nltk
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')
stopword = set(nltk.corpus.stopwords.words('english'))
def remove_stopwords(text):
    word_tokens = word_tokenize(text) 
    text = [word for word in word_tokens if word not in stopword]
    text1=" "
    return (text1.join(text)) 
    
regression_df['title'] = regression_df['title'].apply(lambda x: remove_stopwords(x))

In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

In [ ]:
regression_df["Category"]= regression_df["Category"].replace("Safe To Ignore", 0) 
regression_df["Category"]= regression_df["Category"].replace("Concerned", 1)
regression_df["Category"]= regression_df["Category"].replace("Strongly Concerned", 2)  

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in regression_df.title: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "

In [ ]:
X = tokenizer.texts_to_sequences(regression_df['title'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)
Y = pd.get_dummies(regression_df['Category']).values
print('Shape of label tensor:', Y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

MODEL BUILDING

In [ ]:
inputs = Input(shape = (X.shape[1], ))
embed_layer = Embedding(50000, 264, input_length=X_train.shape[1])(inputs)
embed_layer = SpatialDropout1D(0.2)(embed_layer)
lstm_layer = LSTM(100,dropout=0.2,recurrent_dropout=0.2)(embed_layer)
output = Dense(3,activation='softmax')(lstm_layer)
r_model = Model(inputs = inputs, outputs = output)

In [ ]:
r_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs = 30
batch_size = 64
history = r_model.fit(X_train, y_train,epochs=epochs,batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = r_model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!pip install httplib2==0.15.0

!pip install google-api-python-client==1.6


In [ ]:
r_model.save('rmodel.h5')
model_file = drive.CreateFile({'title' : 'rmodel.h5'})
model_file.SetContentFile('rmodel.h5')
model_file.Upload()

In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

In [ ]:
vect = CountVectorizer(tokenizer=tokenization,ngram_range=(1, 2))
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
rmodel = LogisticRegression(C=0.2, dual=False)
rmodel.fit(tf_train, y_train)
preds = rmodel.predict(tf_test)
accuracy = ((preds == y_test).mean() )*100

In [ ]:
print(accuracy)

In [ ]:
preds

In [ ]:
import joblib
joblib.dump(rmodel, "rmodel.pkl") 

In [ ]:
from google.colab import files
files.download("rmodel.pkl")

In [ ]:
cmodel_file = drive.CreateFile({'title' : 'cmodel.h5'})
cmodel_file.SetContentFile('cmodel.h5')
cmodel_file.Upload()

In [ ]:
rmodel_file = drive.CreateFile({'title' : 'rmodel.h5'})
rmodel_file.SetContentFile('rmodel.h5')
rmodel_file.Upload()

In [ ]:
# download to google drive 
drive.CreateFile({'id': cmodel_file.get('id')})


In [ ]:
# download to google drive 
drive.CreateFile({'id': rmodel_file.get('id')})


In [ ]:
import keras
file_obj = drive.CreateFile({'id': '1hxcutQO7Baw94-mJGjKZcB3GHCXr49RU'})
file_obj.GetContentFile('rmodel.h5')
model = keras.models.load_model('rmodel.h5')